<a href="https://colab.research.google.com/github/darsh7807/learn-llm-with-me/blob/main/llms_from_scratch/pytorch_basicstorch_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.cuda.is_available())


False


In [ ]:
tensor0d= torch.tensor(1)
tensor1d= torch.tensor([1,2,3])
tensor2d= torch.tensor([[1,2,2], [3,4,2]])
tensor3d= torch.tensor( [[[1,2], [1,2]],
                         [[3,4], [5,6]]])
print(tensor0d.to(torch.float64).dtype)
print(tensor2d.shape)
# print(tensor1d.shape)
print(tensor2d.view(3,2))
print(tensor2d.T)
print(tensor2d@tensor2d.T)

torch.float64
torch.Size([2, 3])
tensor([[1, 2],
        [2, 3],
        [4, 2]])
tensor([[1, 3],
        [2, 4],
        [2, 2]])
tensor([[ 9, 15],
        [15, 29]])


In [ ]:
import torch.nn.functional as F

y = torch.tensor([1.0])
x1= torch.tensor([1.1], requires_grad=True)
w1=torch.tensor([2.2], requires_grad=True)
b=torch.tensor([0.0], requires_grad=True)



In [ ]:
z = x1@w1+b
a=torch.sigmoid(z)
loss=F.binary_cross_entropy( a,y)
print(loss)
loss.backward()



tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
class NeuralNetwork(torch.nn.Module):
  def __init__(self, num_inputs, num_outputs):
    super().__init__()
    self.layers  = torch.nn.Sequential(
        #layer 1
        torch.nn.Linear(num_inputs, 30),
        torch.nn.ReLU(),

        #layer 2

        torch.nn.Linear(30,20),
        torch.nn.ReLU(),

        #layer 3
        torch.nn.Linear(20,num_outputs),
    )
  def forward(self, x):
    logits=self.layers(x)
    return logits


In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(50,3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [ ]:
sump=0
for p in model.parameters():
  if p.requires_grad:
    sump=sump+p.numel()

print("num of model param are :", sump)



num of model param are : 2213


In [ ]:
print(model.layers[0].bias.shape)

torch.Size([30])


In [ ]:
torch.manual_seed(123)
x=torch.rand((1,50))
out = model(x)
print(out)

with torch.no_grad():
  out =model(x)
  out = torch.softmax(out, dim=1)
print(out)


tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)
tensor([[0.3113, 0.3934, 0.2952]])


In [ ]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5],
])
y_train  = torch.tensor([
    0,0,0,1,1
])

X_test=torch.tensor([[-0.8, 2.8],
                    [2.6, -1.6]])
y_test = torch.tensor([0,1])


In [ ]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
  def __init__(self, x,y) -> None:
    self.features= x
    self.labels=y

  def __getitem__(self, index):
    one_x=self.features[index]
    one_y=self.labels[index]
    return one_x, one_y
  def __len__(self):
    return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds= ToyDataset(X_test, y_test)
print(len(train_ds))
print(len(test_ds))



5
2


In [ ]:
from torch.utils.data import DataLoader
torch.manual_seed(123)


train_loader=DataLoader(
    dataset = train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    # drop_last=True
)

test_loader=DataLoader(dataset=test_ds,
                     batch_size=2,
                     shuffle=False,
                     num_workers=0)

for idx, (x,y) in enumerate(train_loader):
  print(f"Batch {idx+1}:", x,y)


Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [ ]:
train_loader=DataLoader(
    dataset = train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)
for idx, (x,y) in enumerate(train_loader):
  print(f"Batch {idx+1}:", x,y)


Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


In [ ]:
import torch.nn.functional as F

torch.manual_seed(123)
model= NeuralNetwork(num_inputs=2, num_outputs=2)
optmiser=torch.optim.SGD(
    model.parameters(), lr=0.05
)



In [ ]:
num_epochs=3
for epoch in range(num_epochs):
  model.train()
  for batch_idx, (features, lables) in enumerate(train_loader):
    logits = model(features)
    loss= F.cross_entropy(logits, lables)
    optmiser.zero_grad()
    loss.backward()
    optmiser.step()
    print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
          f"| Batch: {batch_idx:03d}/{len(train_loader):03d}"
          f"| Train Loss: {loss:.2f}")
model.eval()

Epoch: 001/003| Batch: 000/002| Train Loss: 0.75
Epoch: 001/003| Batch: 001/002| Train Loss: 0.60
Epoch: 002/003| Batch: 000/002| Train Loss: 0.61
Epoch: 002/003| Batch: 001/002| Train Loss: 0.58
Epoch: 003/003| Batch: 000/002| Train Loss: 0.48
Epoch: 003/003| Batch: 001/002| Train Loss: 0.41


NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [ ]:
param=0
for layers in model.parameters():
  param = param+ layers.numel()
print(param)

752


In [ ]:
model.eval()
with torch.no_grad():
  outputs = model(X_train)
print(outputs)

tensor([[ 0.3348, -0.8548],
        [ 0.2843, -0.7862],
        [ 0.2147, -0.6908],
        [-0.1854,  0.0968],
        [-0.2144,  0.1194]])


In [ ]:
torch.set_printoptions(sci_mode=False)
probas  = torch.softmax(outputs, dim=-1)
print(probas)
predictions = torch.argmax(probas,dim=1)
print(predictions)

tensor([[0.7667, 0.2333],
        [0.7447, 0.2553],
        [0.7121, 0.2879],
        [0.4299, 0.5701],
        [0.4173, 0.5827]])
tensor([0, 0, 0, 1, 1])


In [ ]:
def compute_accurace(model, dataloader):
  model = model.eval()
  correct = 0.0
  total_example = 0

  for idx, (features, lables ) in enumerate(dataloader):
    with torch.no_grad():
      logits = model(features)

    predictions = torch.argmax(logits, dim=-1)
    compare = predictions == lables
    correct += torch.sum(compare)
    total_example += len(compare)
  return (correct/total_example).item()


print(compute_accurace(model, test_loader))


1.0


In [ ]:
torch.save(model.state_dict(), "model.pth")


In [ ]:
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load("model.pth"))
print(compute_accurace(model, test_loader))



1.0


In [ ]:
print(torch.cuda.is_available())

False


In [ ]:
tensor_1 = torch.tensor([1. , 2., 3.])
tensor_2 = torch.tensor([4., 5.,6.])
print(tensor_1+tensor_2)

tensor_1 = torch.tensor([1. , 2., 3.]).to("cuda")
tensor_2 = torch.tensor([4., 5.,6.]).to("cuda")

print(tensor_1+tensor_2)


tensor([5., 7., 9.])


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
torch.manual_seed(123)
model=NeuralNetwork(2,2)

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimiser = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):
  model.train()
  for idx, (features, lables) in enumerate(train_loader):
    logits = model(features)
    loss = F.cross_entropy(logits, lables)

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    print(f"Eporch: {idx+1:03d}/{num_epochs:03d}"
          f" | Batch {batch_idx:03d}"
          f" | Train/val loss: {loss:.2f}")
    model.eval()

Eporch: 001/003 | Batch 001 | Train/val loss: 0.75
Eporch: 002/003 | Batch 001 | Train/val loss: 0.65
Eporch: 001/003 | Batch 001 | Train/val loss: 0.44
Eporch: 002/003 | Batch 001 | Train/val loss: 0.13
Eporch: 001/003 | Batch 001 | Train/val loss: 0.03
Eporch: 002/003 | Batch 001 | Train/val loss: 0.00
